In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

In [151]:
"""
愚直なHessian vector product積
"""

x = tf.Variable([1., 1.], dtype=tf.float32)
vector = tf.convert_to_tensor([[8], [4]], dtype=tf.float32)


with tf.GradientTape(persistent=True) as t2:
    with tf.GradientTape() as t1:    
        f = x[0]**3 + 2*x[0]*x[1] + x[1] ** 2 - x[0]
    (fx, fy) = t1.gradient(f, x)

fxx, fxy = t2.gradient(fx, x)
fyx, fyy = t2.gradient(fy, x)
H = tf.convert_to_tensor([[fxx, fxy], [fyx, fyy]])

hessian_vector_product = tf.matmul(H, vector) 


invH = tf.linalg.inv(H)
costgrad = tf.convert_to_tensor([[16], [12]], dtype=tf.float32)
invH_grad_product = tf.matmul(invH, costgrad)

del t2
print("H")
print(H.numpy())
print()
print("Hv")
print(hessian_vector_product.numpy())
print()
print()
print("invH")
print(invH.numpy())
print()
print("invH_g_product")
print(invH_grad_product.numpy())

H
[[6. 2.]
 [2. 2.]]

Hv
[[56.]
 [24.]]


invH
[[ 0.25       -0.25000003]
 [-0.25000003  0.75000006]]

invH_g_product
[[0.9999995]
 [5.0000005]]


In [150]:
"""賢いHv product"""

x = tf.Variable([1., 1.], dtype=tf.float32)
vector = tf.convert_to_tensor([[8], [4]], dtype=tf.float32)

with tf.GradientTape() as t2:
    with tf.GradientTape() as t1:    
        f = x[0]**3 + 2*x[0]*x[1] + x[1] ** 2 - x[0]
    grads = t1.gradient(f, x)
    grads_vector_product = tf.reduce_sum([g*v for g, v in zip(grads, vector)])

fast_hessian_vector_product = t2.gradient(grads_vector_product, x)
print("Fast Hv")
print(fast_hessian_vector_product.numpy())

Fast Hv
[56. 24.]


## 演習

共役勾配法でFgを計算する

Fはfのhessian
gは適当

In [23]:
import tensorflow as tf
import numpy as np


def invHg_stupid(theta, f, g):
    """
    　愚直に計算する
    """
    with tf.GradientTape(persistent=True) as t2:
        with tf.GradientTape() as t1:    
            ret = f(theta)
        (f1, f2, f3) = t1.gradient(ret, theta)

    f11, f12, f13 = t2.gradient(f1, theta)
    f21, f22, f23 = t2.gradient(f2, theta)
    f31, f32, f33 = t2.gradient(f3, theta)
    H = tf.convert_to_tensor([[f11, f12, f13], [f21, f22, f23], [f31, f32, f33]])
    
    invF = tf.linalg.inv(H)
    invFg = tf.matmul(invF, g) 
    
    del t2
    return invFg



def invHg_CG(theta, f, b, iters=10):
    """共役勾配法による近似解
    """
    @tf.function
    def f_Ax(x):
        """
            任意のｘに対してAxを計算して返す
        """
        with tf.GradientTape() as t2:
            with tf.GradientTape() as t1:    
                ret = f(theta)
            grads = t1.gradient(ret, theta)
            grads_vector_product = tf.matmul(tf.reshape(grads, [1, -1]), x)
            
        hvp = tf.reshape(t2.gradient(grads_vector_product, theta), shape=[-1, 1])
        
        return hvp + x * 1e-3
    
    x = tf.zeros_like(b)
    r = tf.identity(b)
    p = tf.identity(r)
    r_dot_r = tf.reduce_sum(r*r)
    
    for i in range(iters):
        Ap = f_Ax(p)
        v = r_dot_r / (tf.matmul(tf.transpose(p), Ap))
        x += v*p
        r -= v*Ap
        new_r_dot_r = tf.reduce_sum(r*r)
        mu = new_r_dot_r / r_dot_r
        p = r + mu * p
        r_dot_r = new_r_dot_r
        if r_dot_r < 1e-10:
            break
    
    return x



def f(params):
    return params[0]**3 + 2*params[0]*params[1] + params[1] ** 2 - params[0] + params[2] **2


params = tf.Variable([3., 1., 5.], dtype=tf.float32)
g = tf.convert_to_tensor([[3], [12], [6]], dtype=tf.float32)

print("愚直な計算結果：", invHg_stupid(params, f, g))
print()
print("CGによる近似：", invHg_CG(params, f, g))






愚直な計算結果： tf.Tensor(
[[-0.5625]
 [ 6.5625]
 [ 3.    ]], shape=(3, 1), dtype=float32)

CGによる近似： tf.Tensor(
[[-0.56205505]
 [ 6.5587754 ]
 [ 2.9985006 ]], shape=(3, 1), dtype=float32)
